# Download Steam Images

Reference: https://github.com/woctezuma/steam-DINOv2

## Define functions

Reference: https://github.com/woctezuma/steam-store-snapshots

In [ ]:
import requests

API_URL = "https://api.steampowered.com/ISteamApps/GetAppList/v2"

def get_app_ids(url=API_URL):
  r = requests.get(url)

  if r.ok:
    data = r.json()
    app_ids = { app["appid"] for app in data["applist"]["apps"] }
  else:
    app_ids = {}

  return sorted(app_ids)

In [ ]:
CDN_URL = "https://cdn.cloudflare.steamstatic.com/steam/apps"
IMAGE_NAME = "library_600x900.jpg"

def get_image_url(app_id):
  return f"{CDN_URL}/{app_id}/{IMAGE_NAME}"

In [ ]:
URL_LIST_FNAME = "urls.txt"

def save_list(elements, fname):
  with open(fname, 'w') as f:
    for e in elements:
      f.write(f'{e}\n')

def save_image_list(app_ids, fname=URL_LIST_FNAME):
  elements = [get_image_url(app_id) for app_id in app_ids ]
  save_list(elements, fname)

In [ ]:
import json

APP_LIST_FNAME = "apps.json"
FILTERED_APP_LIST_FNAME = 'filtered_apps.json'

def save_app_list(app_ids, fname=APP_LIST_FNAME):
  with open(fname, 'w') as f:
    json.dump(app_ids, f)

## Prepare app IDs and image urls

In [ ]:
app_ids = get_app_ids()

In [ ]:
save_app_list(app_ids, fname=APP_LIST_FNAME)
save_image_list(app_ids, fname=URL_LIST_FNAME)

## Download images

Reference: https://github.com/rom1504/img2dataset

In [ ]:
%pip install -qq img2dataset

In [ ]:
!img2dataset --url_list=$URL_LIST_FNAME --image_size=224 --resize_mode=keep_ratio

In [ ]:
!du -sh images
!tar czf images_partA.tar.gz images/0000*/
!tar czf images_partB.tar.gz images/0001*/
!du -sh images_part*.tar.gz

## Filter app IDs

In [ ]:
from pathlib import Path

def to_image_index(image_name, image_suffix='.jpg'):
  index_as_str = Path(image_name).name.removesuffix(image_suffix)
  return int(index_as_str)

In [ ]:
import glob

image_names = sorted(glob.glob('images/*/*.jpg'))

In [ ]:
image_indices = [to_image_index(image_name) for image_name in image_names]
filtered_app_ids = [app_ids[index] for index in image_indices]

In [ ]:
save_app_list(filtered_app_ids, fname=FILTERED_APP_LIST_FNAME)